# INCOMPLETE !!!

In [3]:
import albumentations as A # to make holes 
from torchvision.transforms import v2 # for other transforms 

import torch 
import torch.nn as nn 

from PIL import Image 

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
cut_tfm = A.CoarseDropout(num_holes_range=(1,3), p=1)
more_tfms = v2.Compose([
    v2.ToTensor(), 
    v2.ToDtype(torch.float32), 
    v2.RandomHorizontalFlip(p=0.5), 
    v2.RandomAffine(degrees=(-10, 10), scale=(0.9, 1.1))
])
def apply_transform_on_pil_img(pil_img, device=device): 
    return torch.tensor(cut_tfm(image=more_tfms(pil_img).numpy())['image'], device=device) 

c:\Python\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [11]:
# test this out 
a = Image.open('./data/sample_bird_img.jpeg')
apply_transform_on_pil_img(a)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

conv, then deconv 

In [ ]:
class ImageVAE(nn.Module): 
